# Lab 7: Numerical Solution Methods for Dynamic Models
## Value Function Iteration, Time Iteration, and the RBC Model

---

### 🎯 Lab Philosophy

In Lab 6, we explored neural network approaches to solving dynamic models. Now we return to the **classical numerical methods** that form the foundation of computational macroeconomics. These grid-based methods provide:

1. **Guaranteed Convergence**: Under standard assumptions, VFI converges to the unique fixed point.
2. **Transparent Accuracy**: We can directly measure and control approximation errors.
3. **Benchmarks for Validation**: Essential for validating more sophisticated methods.

### 📚 Coverage

**Part 1: The RBC Model and Calibration**
-   Setting up the stochastic Real Business Cycle model.
-   Calibrating parameters to match US business cycle moments.
-   Discretizing the AR(1) productivity process using Tauchen's method.

**Part 2: Value Function Iteration (VFI)**
-   Constructing capital grids (uniform and clustered).
-   Implementing VFI with linear interpolation.
-   Upgrading to cubic spline interpolation for smoother policy functions.

**Part 3: Time Iteration (Euler Equation Methods)**
-   Iterating directly on the consumption policy function.
-   Comparing convergence properties with VFI.

**Part 4: Simulation and Business Cycle Statistics**
-   Simulating model economies.
-   Computing and comparing moments to data.

---

### 🔧 The Research Architect Workflow

Remember our specification-driven approach:
1. **Mathematical Formulation**: Define the Bellman equation and equilibrium conditions.
2. **Algorithmic Design**: Write pseudocode for VFI and time iteration.
3. **Define Deliverables**: Policy functions, convergence diagnostics, simulated moments.
4. **Implementation**: Use AI to generate code from specifications.
5. **Validation**: Check against known solutions and economic intuition.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.optimize import brentq
from numba import njit, prange
import time
from tqdm.auto import tqdm

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully!")

# Part 1: The RBC Model and Calibration

## 1.1 Model Setup

The standard Real Business Cycle model features a representative household maximizing:

$$\max E_0 \sum_{t=0}^{\infty} \beta^t u(c_t)$$

subject to the resource constraint:

$$c_t + k_{t+1} = z_t k_t^\alpha + (1-\delta)k_t$$

where productivity follows an AR(1) process:

$$\log z_{t+1} = \rho \log z_t + \varepsilon_{t+1}, \quad \varepsilon \sim N(0, \sigma^2)$$

### The Bellman Equation

$$V(k, z) = \max_{k'} \left\{ u(c) + \beta E[V(k', z') | z] \right\}$$

where $c = zk^\alpha + (1-\delta)k - k'$.

### The Euler Equation

$$u'(c) = \beta E\left[ u'(c') \left( \alpha z' (k')^{\alpha-1} + 1 - \delta \right) \right]$$

In [ ]:
# =============================================================================
# 1.2 Model Parameters and Calibration
# =============================================================================

class RBCModel:
    """
    Real Business Cycle Model with standard calibration.
    
    Calibration targets US quarterly data:
    - Capital share (alpha): ~0.33 from labor income share data
    - Discount factor (beta): ~0.99 quarterly to match risk-free rate
    - Depreciation (delta): ~0.025 quarterly from BEA data
    - Persistence (rho): ~0.95 from Solow residual estimates
    - Shock volatility (sigma): ~0.007 to match output volatility
    """
    
    def __init__(self, 
                 alpha=0.33,      # Capital share
                 beta=0.99,       # Discount factor (quarterly)
                 delta=0.025,     # Depreciation rate (quarterly)
                 sigma_crra=1.0,  # CRRA coefficient (log utility)
                 rho=0.95,        # Persistence of productivity
                 sigma_eps=0.007, # Std dev of productivity shocks
                 n_z=7,           # Number of productivity grid points
                 n_k=100):        # Number of capital grid points
        
        # Store parameters
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.sigma_crra = sigma_crra
        self.rho = rho
        self.sigma_eps = sigma_eps
        self.n_z = n_z
        self.n_k = n_k
        
        # Compute steady state
        self._compute_steady_state()
        
        # Discretize productivity process
        self._discretize_productivity()
        
        # Construct capital grid
        self._construct_capital_grid()
        
    def _compute_steady_state(self):
        """Compute deterministic steady state values."""
        # From FOC: r = 1/beta - (1-delta) = alpha * k^(alpha-1)
        r_ss = 1/self.beta - (1 - self.delta)
        self.k_ss = (self.alpha / r_ss) ** (1 / (1 - self.alpha))
        self.y_ss = self.k_ss ** self.alpha
        self.c_ss = self.y_ss - self.delta * self.k_ss
        self.i_ss = self.delta * self.k_ss
        
        print("=== Steady State Values ===")
        print(f"Capital (k*):     {self.k_ss:.4f}")
        print(f"Output (y*):      {self.y_ss:.4f}")
        print(f"Consumption (c*): {self.c_ss:.4f}")
        print(f"Investment (i*):  {self.i_ss:.4f}")
        print(f"K/Y ratio:        {self.k_ss/self.y_ss:.4f}")
        
    def _discretize_productivity(self):
        """
        Discretize AR(1) process using Tauchen's method.
        
        The unconditional std dev of log(z) is sigma_eps / sqrt(1 - rho^2).
        We cover +/- 3 std devs.
        """
        sigma_z = self.sigma_eps / np.sqrt(1 - self.rho**2)
        z_max = 3 * sigma_z
        z_min = -z_max
        
        # Grid for log(z)
        log_z_grid = np.linspace(z_min, z_max, self.n_z)
        step = log_z_grid[1] - log_z_grid[0]
        
        # Productivity grid (levels)
        self.z_grid = np.exp(log_z_grid)
        self.log_z_grid = log_z_grid
        
        # Transition matrix using Tauchen's method
        self.Pi = np.zeros((self.n_z, self.n_z))
        
        for i in range(self.n_z):
            for j in range(self.n_z):
                if j == 0:
                    self.Pi[i, j] = self._std_normal_cdf(
                        (log_z_grid[0] + step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    )
                elif j == self.n_z - 1:
                    self.Pi[i, j] = 1 - self._std_normal_cdf(
                        (log_z_grid[-1] - step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    )
                else:
                    self.Pi[i, j] = self._std_normal_cdf(
                        (log_z_grid[j] + step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    ) - self._std_normal_cdf(
                        (log_z_grid[j] - step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    )
        
        print(f"\n=== Productivity Grid ===")
        print(f"z ranges from {self.z_grid[0]:.4f} to {self.z_grid[-1]:.4f}")
        print(f"Steady state z = 1.0 (approx. middle of grid)")
        
    def _std_normal_cdf(self, x):
        """Standard normal CDF using error function."""
        from scipy.special import erfc
        return 0.5 * erfc(-x / np.sqrt(2))
    
    def _construct_capital_grid(self, grid_type='clustered'):
        """
        Construct capital grid.
        
        Options:
        - 'uniform': Equally spaced points
        - 'clustered': More points near steady state (recommended)
        """
        k_min = 0.5 * self.k_ss  # 50% below steady state
        k_max = 1.5 * self.k_ss  # 50% above steady state
        
        if grid_type == 'uniform':
            self.k_grid = np.linspace(k_min, k_max, self.n_k)
        elif grid_type == 'clustered':
            # Use a transformation to cluster points near steady state
            # x in [0,1] -> k via sinh transformation
            x = np.linspace(0, 1, self.n_k)
            # Sinh-based clustering (more points in middle)
            stretch = 2.0  # Controls clustering intensity
            x_transformed = np.sinh(stretch * (x - 0.5)) / np.sinh(stretch * 0.5)
            x_transformed = (x_transformed + 1) / 2  # Map back to [0,1]
            self.k_grid = k_min + (k_max - k_min) * x_transformed
        
        print(f"\n=== Capital Grid ===")
        print(f"k ranges from {self.k_grid[0]:.4f} to {self.k_grid[-1]:.4f}")
        print(f"Number of grid points: {self.n_k}")
        print(f"Grid type: {grid_type}")
        
    def utility(self, c):
        """CRRA utility function."""
        if self.sigma_crra == 1.0:
            return np.log(np.maximum(c, 1e-10))
        else:
            return (np.maximum(c, 1e-10) ** (1 - self.sigma_crra) - 1) / (1 - self.sigma_crra)
    
    def marginal_utility(self, c):
        """Marginal utility."""
        return np.maximum(c, 1e-10) ** (-self.sigma_crra)
    
    def inv_marginal_utility(self, mu):
        """Inverse marginal utility."""
        return mu ** (-1 / self.sigma_crra)
    
    def production(self, k, z):
        """Production function."""
        return z * k ** self.alpha
    
    def marginal_product_k(self, k, z):
        """Marginal product of capital."""
        return self.alpha * z * k ** (self.alpha - 1)

In [ ]:
# Initialize the model
model = RBCModel(n_k=100, n_z=7)

# Visualize the capital grid
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Capital grid visualization
axes[0].plot(model.k_grid, np.ones(model.n_k), 'b|', markersize=15)
axes[0].axvline(model.k_ss, color='r', linestyle='--', label=f'Steady State k* = {model.k_ss:.2f}')
axes[0].set_xlabel('Capital')
axes[0].set_title('Capital Grid (Clustered near Steady State)')
axes[0].legend()
axes[0].set_yticks([])

# Transition matrix heatmap
im = axes[1].imshow(model.Pi, cmap='Blues', aspect='auto')
axes[1].set_xlabel('z\' (next period)')
axes[1].set_ylabel('z (current period)')
axes[1].set_title('Productivity Transition Matrix (Tauchen)')
plt.colorbar(im, ax=axes[1])

plt.tight_layout()
plt.show()

# Part 2: Value Function Iteration (VFI)

## 2.1 The VFI Algorithm

**Pseudocode:**
```
1. Initialize V_0(k, z) = 0 for all (k, z)
2. For iteration n = 1, 2, ...
   a. For each state (k_i, z_j):
      - For each possible k' on grid:
        * Compute c = z*k^α + (1-δ)k - k'
        * If c > 0: compute u(c) + β * E[V_{n-1}(k', z') | z]
      - V_n(k_i, z_j) = max over k' choices
      - Store optimal k' in policy function
   b. Check convergence: ||V_n - V_{n-1}|| < tolerance
3. Return converged V and policy function g(k, z)
```

**Key Implementation Choices:**
- **Interpolation**: Linear vs. cubic spline for evaluating V at off-grid k' values
- **Optimization**: Grid search vs. golden section vs. Brent's method
- **Acceleration**: Howard's policy improvement (covered in Lab 8)

In [ ]:
# =============================================================================
# 2.2 VFI Implementation with Linear Interpolation
# =============================================================================

class VFISolver:
    """
    Value Function Iteration solver for the RBC model.
    """
    
    def __init__(self, model, interpolation='linear'):
        self.model = model
        self.interpolation = interpolation
        
        # Initialize value function and policy
        self.V = np.zeros((model.n_k, model.n_z))
        self.policy_k = np.zeros((model.n_k, model.n_z))
        self.policy_c = np.zeros((model.n_k, model.n_z))
        
        # Convergence history
        self.convergence_history = []
        
    def _interpolate_V(self, k_prime, z_idx):
        """
        Interpolate value function at k' for given z state.
        """
        if self.interpolation == 'linear':
            return np.interp(k_prime, self.model.k_grid, self.V[:, z_idx])
        elif self.interpolation == 'cubic':
            # Create cubic spline (with boundary conditions)
            spline = interpolate.CubicSpline(
                self.model.k_grid, self.V[:, z_idx], bc_type='natural'
            )
            return spline(k_prime)
        
    def _compute_expected_V(self, k_prime, z_idx):
        """
        Compute E[V(k', z') | z] = sum over z' of Pi(z, z') * V(k', z').
        """
        expected_V = 0.0
        for z_prime_idx in range(self.model.n_z):
            V_kprime_zprime = self._interpolate_V(k_prime, z_prime_idx)
            expected_V += self.model.Pi[z_idx, z_prime_idx] * V_kprime_zprime
        return expected_V
    
    def _bellman_rhs(self, k_prime, k, z, z_idx):
        """
        Compute right-hand side of Bellman equation for given (k, z, k').
        Returns -infinity if consumption is non-positive.
        """
        # Resources available
        resources = z * k**self.model.alpha + (1 - self.model.delta) * k
        c = resources - k_prime
        
        if c <= 0:
            return -np.inf
        
        # Current utility + discounted expected continuation value
        u = self.model.utility(c)
        EV = self._compute_expected_V(k_prime, z_idx)
        
        return u + self.model.beta * EV
    
    def solve(self, tol=1e-6, max_iter=1000, verbose=True):
        """
        Solve for value function and policy using VFI.
        """
        print(f"\n=== Starting VFI ({self.interpolation} interpolation) ===")
        start_time = time.time()
        
        for iteration in range(max_iter):
            V_old = self.V.copy()
            
            # Loop over all states
            for i_k, k in enumerate(self.model.k_grid):
                for i_z, z in enumerate(self.model.z_grid):
                    
                    # Maximum resources (upper bound for k')
                    max_resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                    k_prime_max = min(max_resources - 1e-8, self.model.k_grid[-1])
                    k_prime_min = self.model.k_grid[0]
                    
                    # Grid search over k'
                    # (In practice, we'd use optimization, but grid search is clearer)
                    best_val = -np.inf
                    best_k_prime = k_prime_min
                    
                    for k_prime in self.model.k_grid:
                        if k_prime > k_prime_max:
                            break
                        val = self._bellman_rhs(k_prime, k, z, i_z)
                        if val > best_val:
                            best_val = val
                            best_k_prime = k_prime
                    
                    # Update value and policy
                    self.V[i_k, i_z] = best_val
                    self.policy_k[i_k, i_z] = best_k_prime
                    self.policy_c[i_k, i_z] = max_resources - best_k_prime
            
            # Check convergence
            diff = np.max(np.abs(self.V - V_old))
            self.convergence_history.append(diff)
            
            if verbose and iteration % 50 == 0:
                print(f"Iteration {iteration:4d}: ||V - V_old|| = {diff:.2e}")
            
            if diff < tol:
                elapsed = time.time() - start_time
                print(f"\nConverged in {iteration+1} iterations ({elapsed:.2f} seconds)")
                return True
        
        print(f"\nWARNING: Did not converge after {max_iter} iterations")
        return False
    
    def plot_results(self):
        """Visualize value function and policy functions."""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Plot for different productivity states
        z_indices = [0, self.model.n_z // 2, self.model.n_z - 1]
        colors = ['blue', 'green', 'red']
        labels = ['Low z', 'Medium z', 'High z']
        
        # Value Function
        for idx, (i_z, color, label) in enumerate(zip(z_indices, colors, labels)):
            axes[0, 0].plot(self.model.k_grid, self.V[:, i_z], 
                           color=color, label=f'{label} (z={self.model.z_grid[i_z]:.2f})')
        axes[0, 0].axvline(self.model.k_ss, color='gray', linestyle='--', alpha=0.5)
        axes[0, 0].set_xlabel('Capital (k)')
        axes[0, 0].set_ylabel('V(k, z)')
        axes[0, 0].set_title('Value Function')
        axes[0, 0].legend()
        
        # Capital Policy
        for idx, (i_z, color, label) in enumerate(zip(z_indices, colors, labels)):
            axes[0, 1].plot(self.model.k_grid, self.policy_k[:, i_z], 
                           color=color, label=label)
        axes[0, 1].plot(self.model.k_grid, self.model.k_grid, 'k--', alpha=0.3, label='45° line')
        axes[0, 1].axvline(self.model.k_ss, color='gray', linestyle='--', alpha=0.5)
        axes[0, 1].set_xlabel('Capital (k)')
        axes[0, 1].set_ylabel("k'")
        axes[0, 1].set_title('Capital Policy Function')
        axes[0, 1].legend()
        
        # Consumption Policy
        for idx, (i_z, color, label) in enumerate(zip(z_indices, colors, labels)):
            axes[1, 0].plot(self.model.k_grid, self.policy_c[:, i_z], 
                           color=color, label=label)
        axes[1, 0].axvline(self.model.k_ss, color='gray', linestyle='--', alpha=0.5)
        axes[1, 0].set_xlabel('Capital (k)')
        axes[1, 0].set_ylabel('c')
        axes[1, 0].set_title('Consumption Policy Function')
        axes[1, 0].legend()
        
        # Convergence
        axes[1, 1].semilogy(self.convergence_history)
        axes[1, 1].set_xlabel('Iteration')
        axes[1, 1].set_ylabel('||V - V_old||')
        axes[1, 1].set_title('Convergence History')
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Solve with linear interpolation
vfi_linear = VFISolver(model, interpolation='linear')
vfi_linear.solve(tol=1e-3, max_iter=500)
vfi_linear.plot_results()

## 2.3 Cubic Spline Interpolation

Linear interpolation can introduce kinks in the policy function. Cubic splines provide:
- Smoother approximations
- Better accuracy with fewer grid points
- Continuous first derivatives (important for Euler equation accuracy)

In [ ]:
# Solve with cubic spline interpolation
vfi_cubic = VFISolver(model, interpolation='cubic')
vfi_cubic.solve(tol=1e-1, max_iter=50)
vfi_cubic.plot_results()

In [ ]:
# Compare linear vs cubic policy functions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

i_z_mid = model.n_z // 2

# Capital policy comparison
axes[0].plot(model.k_grid, vfi_linear.policy_k[:, i_z_mid], 'b-', 
             label='Linear Interpolation', linewidth=2)
axes[0].plot(model.k_grid, vfi_cubic.policy_k[:, i_z_mid], 'r--', 
             label='Cubic Spline', linewidth=2)
axes[0].set_xlabel('Capital (k)')
axes[0].set_ylabel("k'")
axes[0].set_title('Capital Policy: Linear vs Cubic')
axes[0].legend()

# Difference
diff = vfi_cubic.policy_k[:, i_z_mid] - vfi_linear.policy_k[:, i_z_mid]
axes[1].plot(model.k_grid, diff * 100, 'g-', linewidth=2)
axes[1].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Capital (k)')
axes[1].set_ylabel('Difference (%)')
axes[1].set_title('Policy Difference (Cubic - Linear)')

plt.tight_layout()
plt.show()

# Part 3: Time Iteration (Euler Equation Method)

## 3.1 The Time Iteration Algorithm

Instead of iterating on the value function, we iterate directly on the policy function using the Euler equation:

$$u'(c(k, z)) = \beta E\left[ u'(c(k', z')) \cdot R(k', z') \right]$$

where $R(k', z') = \alpha z' (k')^{\alpha-1} + 1 - \delta$ is the gross return on capital.

**Pseudocode:**
```
1. Initialize c_0(k, z) (e.g., fraction of income)
2. For iteration n = 1, 2, ...
   a. For each state (k, z):
      - Compute implied k' from budget: k' = z*k^α + (1-δ)k - c_{n-1}(k,z)
      - Compute expected RHS of Euler:
        E[u'(c_{n-1}(k', z')) * R(k', z')]
      - Invert to get new consumption:
        c_n(k, z) = (u')^{-1}(β * RHS)
   b. Check convergence: ||c_n - c_{n-1}|| < tolerance
3. Return converged consumption policy
```

**Advantages of Time Iteration:**
- Often converges faster than VFI (linear vs. geometric)
- Directly works with the economically meaningful policy
- Euler equation errors are natural accuracy diagnostics

In [ ]:
# =============================================================================
# 3.2 Time Iteration Implementation
# =============================================================================

class TimeIterationSolver:
    """
    Time Iteration solver using the Euler equation.
    """
    
    def __init__(self, model):
        self.model = model
        
        # Initialize consumption policy (fraction of available resources)
        self.policy_c = np.zeros((model.n_k, model.n_z))
        for i_k, k in enumerate(model.k_grid):
            for i_z, z in enumerate(model.z_grid):
                resources = z * k**model.alpha + (1 - model.delta) * k
                self.policy_c[i_k, i_z] = 0.5 * resources  # Initial guess: consume half
        
        # Convergence history
        self.convergence_history = []
        
    def _interpolate_c(self, k_prime, z_idx):
        """Interpolate consumption policy at k' for given z state."""
        return np.interp(k_prime, self.model.k_grid, self.policy_c[:, z_idx])
    
    def _compute_euler_rhs(self, k, z, z_idx, c):
        """
        Compute the right-hand side of the Euler equation:
        β * E[u'(c') * R']
        """
        # Implied next-period capital
        resources = z * k**self.model.alpha + (1 - self.model.delta) * k
        k_prime = resources - c
        
        if k_prime <= self.model.k_grid[0] or k_prime >= self.model.k_grid[-1]:
            # Out of bounds - return something that discourages this
            return np.inf
        
        # Expected marginal utility * return
        euler_rhs = 0.0
        for z_prime_idx in range(self.model.n_z):
            z_prime = self.model.z_grid[z_prime_idx]
            c_prime = self._interpolate_c(k_prime, z_prime_idx)
            
            # Gross return on capital
            R_prime = self.model.alpha * z_prime * k_prime**(self.model.alpha - 1) + (1 - self.model.delta)
            
            # Marginal utility
            mu_prime = self.model.marginal_utility(c_prime)
            
            euler_rhs += self.model.Pi[z_idx, z_prime_idx] * mu_prime * R_prime
        
        return self.model.beta * euler_rhs
    
    def solve(self, tol=1e-8, max_iter=1000, verbose=True):
        """
        Solve for consumption policy using time iteration.
        """
        print("\n=== Starting Time Iteration ===")
        start_time = time.time()
        
        for iteration in range(max_iter):
            policy_c_old = self.policy_c.copy()
            
            # Update consumption policy
            for i_k, k in enumerate(self.model.k_grid):
                for i_z, z in enumerate(self.model.z_grid):
                    resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                    
                    # Compute Euler RHS with current policy
                    euler_rhs = self._compute_euler_rhs(k, z, i_z, policy_c_old[i_k, i_z])
                    
                    if euler_rhs == np.inf or euler_rhs <= 0:
                        # Keep old policy if out of bounds
                        continue
                    
                    # New consumption from inverted Euler
                    c_new = self.model.inv_marginal_utility(euler_rhs)
                    
                    # Ensure feasibility
                    c_new = np.clip(c_new, 1e-10, resources - self.model.k_grid[0])
                    
                    self.policy_c[i_k, i_z] = c_new
            
            # Check convergence
            diff = np.max(np.abs(self.policy_c - policy_c_old))
            self.convergence_history.append(diff)
            
            if verbose and iteration % 50 == 0:
                print(f"Iteration {iteration:4d}: ||c - c_old|| = {diff:.2e}")
            
            if diff < tol:
                elapsed = time.time() - start_time
                print(f"\nConverged in {iteration+1} iterations ({elapsed:.2f} seconds)")
                return True
        
        print(f"\nWARNING: Did not converge after {max_iter} iterations")
        return False
    
    def get_capital_policy(self):
        """Compute capital policy from consumption policy."""
        policy_k = np.zeros_like(self.policy_c)
        for i_k, k in enumerate(self.model.k_grid):
            for i_z, z in enumerate(self.model.z_grid):
                resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                policy_k[i_k, i_z] = resources - self.policy_c[i_k, i_z]
        return policy_k
    
    def plot_results(self):
        """Visualize results."""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        z_indices = [0, self.model.n_z // 2, self.model.n_z - 1]
        colors = ['blue', 'green', 'red']
        labels = ['Low z', 'Medium z', 'High z']
        
        policy_k = self.get_capital_policy()
        
        # Consumption Policy
        for i_z, color, label in zip(z_indices, colors, labels):
            axes[0].plot(self.model.k_grid, self.policy_c[:, i_z], 
                        color=color, label=label)
        axes[0].axvline(self.model.k_ss, color='gray', linestyle='--', alpha=0.5)
        axes[0].set_xlabel('Capital (k)')
        axes[0].set_ylabel('c')
        axes[0].set_title('Consumption Policy (Time Iteration)')
        axes[0].legend()
        
        # Capital Policy
        for i_z, color, label in zip(z_indices, colors, labels):
            axes[1].plot(self.model.k_grid, policy_k[:, i_z], 
                        color=color, label=label)
        axes[1].plot(self.model.k_grid, self.model.k_grid, 'k--', alpha=0.3, label='45° line')
        axes[1].axvline(self.model.k_ss, color='gray', linestyle='--', alpha=0.5)
        axes[1].set_xlabel('Capital (k)')
        axes[1].set_ylabel("k'")
        axes[1].set_title('Capital Policy (Time Iteration)')
        axes[1].legend()
        
        # Convergence
        axes[2].semilogy(self.convergence_history)
        axes[2].set_xlabel('Iteration')
        axes[2].set_ylabel('||c - c_old||')
        axes[2].set_title('Convergence History')
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Solve using Time Iteration
ti_solver = TimeIterationSolver(model)
ti_solver.solve(tol=1e-6, max_iter=500)
ti_solver.plot_results()

In [ ]:
# Compare VFI and Time Iteration policies
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

i_z_mid = model.n_z // 2
ti_policy_k = ti_solver.get_capital_policy()

# Capital policy comparison
axes[0].plot(model.k_grid, vfi_cubic.policy_k[:, i_z_mid], 'b-', 
             label='VFI (Cubic)', linewidth=2)
axes[0].plot(model.k_grid, ti_policy_k[:, i_z_mid], 'r--', 
             label='Time Iteration', linewidth=2)
axes[0].set_xlabel('Capital (k)')
axes[0].set_ylabel("k'")
axes[0].set_title('Capital Policy: VFI vs Time Iteration')
axes[0].legend()

# Consumption policy comparison
axes[1].plot(model.k_grid, vfi_cubic.policy_c[:, i_z_mid], 'b-', 
             label='VFI (Cubic)', linewidth=2)
axes[1].plot(model.k_grid, ti_solver.policy_c[:, i_z_mid], 'r--', 
             label='Time Iteration', linewidth=2)
axes[1].set_xlabel('Capital (k)')
axes[1].set_ylabel('c')
axes[1].set_title('Consumption Policy: VFI vs Time Iteration')
axes[1].legend()

plt.tight_layout()
plt.show()

# Part 4: Simulation and Business Cycle Statistics

## 4.1 Model Simulation

Now we simulate the solved model to:
1. Generate artificial time series for output, consumption, investment
2. Compute second moments (standard deviations, correlations)
3. Compare with US business cycle facts

**US Business Cycle Facts (HP-filtered, quarterly):**
- σ(Y) ≈ 1.5-2.0%
- σ(C)/σ(Y) ≈ 0.5-0.8 (consumption less volatile)
- σ(I)/σ(Y) ≈ 3.0-4.0 (investment more volatile)
- corr(C, Y) ≈ 0.8-0.9
- corr(I, Y) ≈ 0.8-0.9
- Autocorrelation of Y ≈ 0.8-0.9

In [ ]:
# =============================================================================
# 4.2 Simulation Engine
# =============================================================================

class RBCSimulator:
    """
    Simulator for the solved RBC model.
    """
    
    def __init__(self, model, policy_c):
        self.model = model
        self.policy_c = policy_c
        
    def _interpolate_c(self, k, z_idx):
        """Interpolate consumption at given capital level."""
        return np.interp(k, self.model.k_grid, self.policy_c[:, z_idx])
    
    def simulate(self, T=1000, burn_in=200, seed=42):
        """
        Simulate the model for T periods.
        
        Returns dictionary with time series for Y, C, I, K.
        """
        np.random.seed(seed)
        
        T_total = T + burn_in
        
        # Initialize storage
        k_path = np.zeros(T_total + 1)
        z_idx_path = np.zeros(T_total, dtype=int)
        y_path = np.zeros(T_total)
        c_path = np.zeros(T_total)
        i_path = np.zeros(T_total)
        
        # Initial conditions
        k_path[0] = self.model.k_ss
        z_idx_path[0] = self.model.n_z // 2  # Start at median productivity
        
        # Simulate
        for t in range(T_total):
            k = k_path[t]
            z_idx = z_idx_path[t]
            z = self.model.z_grid[z_idx]
            
            # Output
            y = z * k ** self.model.alpha
            y_path[t] = y
            
            # Consumption (from policy)
            c = self._interpolate_c(k, z_idx)
            c_path[t] = c
            
            # Investment and next-period capital
            resources = y + (1 - self.model.delta) * k
            k_prime = resources - c
            k_path[t + 1] = np.clip(k_prime, self.model.k_grid[0], self.model.k_grid[-1])
            
            # Investment
            i_path[t] = k_path[t + 1] - (1 - self.model.delta) * k
            
            # Draw next productivity state
            if t < T_total - 1:
                z_idx_path[t + 1] = np.random.choice(
                    self.model.n_z, p=self.model.Pi[z_idx, :]
                )
        
        # Discard burn-in period
        return {
            'Y': y_path[burn_in:],
            'C': c_path[burn_in:],
            'I': i_path[burn_in:],
            'K': k_path[burn_in:-1],
            'z_idx': z_idx_path[burn_in:]
        }
    
    def compute_moments(self, data, hp_filter=True, hp_lambda=1600):
        """
        Compute business cycle moments from simulated data.
        
        If hp_filter=True, applies HP filter first.
        """
        from scipy.signal import filtfilt
        
        # Log the series
        log_Y = np.log(data['Y'])
        log_C = np.log(data['C'])
        log_I = np.log(np.maximum(data['I'], 1e-10))  # Handle potential negative investment
        
        if hp_filter:
            # Simple HP filter implementation
            def hp_filter_func(y, lam=1600):
                T = len(y)
                # Create penalty matrix
                D = np.zeros((T-2, T))
                for i in range(T-2):
                    D[i, i] = 1
                    D[i, i+1] = -2
                    D[i, i+2] = 1
                # Trend component
                I = np.eye(T)
                trend = np.linalg.solve(I + lam * D.T @ D, y)
                cycle = y - trend
                return cycle, trend
            
            cycle_Y, _ = hp_filter_func(log_Y, hp_lambda)
            cycle_C, _ = hp_filter_func(log_C, hp_lambda)
            cycle_I, _ = hp_filter_func(log_I, hp_lambda)
        else:
            # Simple detrending
            cycle_Y = log_Y - np.mean(log_Y)
            cycle_C = log_C - np.mean(log_C)
            cycle_I = log_I - np.mean(log_I)
        
        # Compute moments
        moments = {
            'std_Y': np.std(cycle_Y) * 100,
            'std_C': np.std(cycle_C) * 100,
            'std_I': np.std(cycle_I) * 100,
            'rel_std_C': np.std(cycle_C) / np.std(cycle_Y),
            'rel_std_I': np.std(cycle_I) / np.std(cycle_Y),
            'corr_CY': np.corrcoef(cycle_C, cycle_Y)[0, 1],
            'corr_IY': np.corrcoef(cycle_I, cycle_Y)[0, 1],
            'autocorr_Y': np.corrcoef(cycle_Y[:-1], cycle_Y[1:])[0, 1]
        }
        
        return moments
    
    def print_moments_comparison(self, moments):
        """Print moments comparison with US data."""
        print("\n" + "="*60)
        print("Business Cycle Moments: Model vs US Data")
        print("="*60)
        print(f"{'Moment':<25} {'Model':>12} {'US Data':>12}")
        print("-"*60)
        print(f"{'σ(Y) (%)':.<25} {moments['std_Y']:>12.2f} {'1.5-2.0':>12}")
        print(f"{'σ(C)/σ(Y)':.<25} {moments['rel_std_C']:>12.2f} {'0.5-0.8':>12}")
        print(f"{'σ(I)/σ(Y)':.<25} {moments['rel_std_I']:>12.2f} {'3.0-4.0':>12}")
        print(f"{'corr(C,Y)':.<25} {moments['corr_CY']:>12.2f} {'0.8-0.9':>12}")
        print(f"{'corr(I,Y)':.<25} {moments['corr_IY']:>12.2f} {'0.8-0.9':>12}")
        print(f"{'autocorr(Y)':.<25} {moments['autocorr_Y']:>12.2f} {'0.8-0.9':>12}")
        print("="*60)

In [ ]:
# Simulate the model
simulator = RBCSimulator(model, ti_solver.policy_c)
sim_data = simulator.simulate(T=2000, burn_in=500)

# Compute moments
moments = simulator.compute_moments(sim_data)
simulator.print_moments_comparison(moments)

In [ ]:
# Visualize simulated time series
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

T_plot = 200  # Plot first 200 periods

# Output
axes[0, 0].plot(sim_data['Y'][:T_plot], 'b-', linewidth=1)
axes[0, 0].axhline(model.y_ss, color='r', linestyle='--', label=f'Steady State = {model.y_ss:.2f}')
axes[0, 0].set_xlabel('Period')
axes[0, 0].set_ylabel('Output')
axes[0, 0].set_title('Simulated Output')
axes[0, 0].legend()

# Consumption
axes[0, 1].plot(sim_data['C'][:T_plot], 'g-', linewidth=1)
axes[0, 1].axhline(model.c_ss, color='r', linestyle='--', label=f'Steady State = {model.c_ss:.2f}')
axes[0, 1].set_xlabel('Period')
axes[0, 1].set_ylabel('Consumption')
axes[0, 1].set_title('Simulated Consumption')
axes[0, 1].legend()

# Investment
axes[1, 0].plot(sim_data['I'][:T_plot], 'm-', linewidth=1)
axes[1, 0].axhline(model.i_ss, color='r', linestyle='--', label=f'Steady State = {model.i_ss:.2f}')
axes[1, 0].set_xlabel('Period')
axes[1, 0].set_ylabel('Investment')
axes[1, 0].set_title('Simulated Investment')
axes[1, 0].legend()

# Capital
axes[1, 1].plot(sim_data['K'][:T_plot], 'c-', linewidth=1)
axes[1, 1].axhline(model.k_ss, color='r', linestyle='--', label=f'Steady State = {model.k_ss:.2f}')
axes[1, 1].set_xlabel('Period')
axes[1, 1].set_ylabel('Capital')
axes[1, 1].set_title('Simulated Capital Stock')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Summary and Key Takeaways

## What We Learned

1. **RBC Model Setup**: Calibrating parameters to match US data, computing steady states, and discretizing continuous processes.

2. **Value Function Iteration**: A guaranteed convergence method that is intuitive but can be slow. Interpolation choice matters for accuracy.

3. **Time Iteration**: Often faster convergence by working directly on policy functions. Natural connection to Euler equation errors.

4. **Simulation**: Connecting solved models to data through moment comparison.

## Looking Ahead (Lab 8)

Next lab we will cover:
- **Howard's Policy Improvement**: Accelerating VFI by 10-100x
- **Endogenous Grid Method (EGM)**: Eliminating the inner optimization loop
- **Euler Equation Errors**: Rigorous accuracy assessment
- **Sensitivity Analysis**: How do results depend on numerical choices?

---

## 🎯 Research Architect Reflection

In this lab, we:
1. **Specified** the mathematical problem (Bellman equation, Euler equation)
2. **Designed** clear algorithms (VFI, Time Iteration pseudocode)
3. **Defined** our deliverables (policy functions, convergence plots, moments)
4. **Implemented** with AI assistance
5. **Validated** by comparing methods and checking against theory

The key insight: your economic understanding (choosing calibration, understanding convergence, knowing what moments matter) is what makes AI-generated code useful.